In [16]:
import requests
import sqlite3
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?locations=TH%2F-EU&name_desc=false"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all country names, years, and GDP per capita
    country_names = soup.find_all('a', class_='country-name')
    values = soup.find_all('div', class_='item')

    # Connect to the SQLite database
    conn = sqlite3.connect('scraped_data.db')
    cursor = conn.cursor()

    # Create SQL table to store the data
    cursor.execute('''CREATE TABLE IF NOT EXISTS scraped_data
                      (country TEXT, year INTEGER, gdp_per_capita REAL)''')

    # Extract data for each country and insert into SQL table
    for i in range(len(country_names)):
        country = country_names[i].text.strip()
        year = int(values[i*2].text.strip())
        gdp_per_capita = float(values[i*2 + 1].text.strip().replace(',', ''))

        # Insert data into SQL table
        cursor.execute("INSERT INTO scraped_data (country, year, gdp_per_capita) VALUES (?, ?, ?)", (country, year, gdp_per_capita))

    # Commit changes and close connection
    conn.commit()
    conn.close()

    print("Data stored successfully in SQL database.")
else:
    print("Failed to retrieve data from the website.")



Data stored successfully in SQL database.


In [17]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('scraped_data.db')
cursor = conn.cursor()

# Execute a SQL query to fetch all rows from the table
cursor.execute("SELECT * FROM scraped_data")

# Fetch all rows from the result set
rows = cursor.fetchall()

# Print the fetched rows
for row in rows:
    print(row)

# Close connection
conn.close()